Advent of Code 2021 - Day 12: Passage Pathing

Part 1:
How many paths through this cave system are there that visit small caves at most once?

In [4]:
from collections import Counter

In [6]:
test_data = """fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW"""

# test_data = """dc-end
# HN-start
# start-kj
# dc-start
# dc-HN
# LN-dc
# HN-end
# kj-sa
# kj-HN
# kj-dc"""


test_data1 = """start-A
start-b
A-c
A-b
b-d
A-end
b-end"""

test_data = test_data.split('\n')
test_data1 = test_data1.split('\n')

In [3]:
with open('input_day12.txt', 'rt') as f:
    data = f.readlines()

data = [item.rstrip() for item in data]

In [ ]:
def find_ways(path, remaining_links):

    start_point = path[-1]

    relevant_links = [link for link in remaining_links if start_point in link]
    
    options = []
    for link in relevant_links:
        points = link.split('-')

        points.remove(start_point)
        new_point = points[0]

        if new_point == 'end':
            possible_links = None
        else:
            possible_links = remaining_links.copy()

            if start_point == 'start':
                possible_links = [link for link in possible_links if 'start' not in link]

        new_path = path.copy()
        new_path.append(new_point)
        options.append((new_path, possible_links))
    
    return options



In [ ]:
# test
ways = find_ways(['start'], test_data.copy())
for way in ways:
    print(way[0], len(way[1]))

In [ ]:
options = [(['start'], data.copy())]

def find_paths(initial_options):

    new_options = []
    for n, option in enumerate(initial_options.copy()):
        # print('n =',n)

        if option[1] is None:   # if the path has not ended
            continue

        new_option = find_ways(option[0], option[1])
        
        new_option = filter_options(new_option)
        
        new_options.extend(new_option)
        initial_options.remove(option)
    
    initial_options.extend(new_options)

    paths, possibilities = zip(*initial_options)
    # print('paths:\n', list(paths))
    print('# of paths:', len(paths))

    if all(possibilities) == None or len(new_options) == 0:
        # print(initial_options)
        return initial_options
    else:
        find_paths(initial_options)


def filter_options(options):

    for path, pos in options.copy():
        for cave in path:
            if cave.islower() and path.count(cave) > 1:
                options.remove((path, pos))
                break
    
    return options


# paths = [item[0] for item in find_paths(options)]
find_paths(options)

In [ ]:
paths, _ = zip(*options)
for path in paths:
    print(path)

In [ ]:
# Part 1 answer
print(len(paths))

Part 2: 

In [ ]:

options = [(['start'], test_data1.copy())]

def find_ways(path, remaining_links):

    start_point = path[-1]

    relevant_links = [link.split('-') for link in remaining_links]
    relevant_links = [link for link in relevant_links if start_point in link]
    
    options = []
    for link in relevant_links:
        
        link.remove(start_point)
        new_point = link[0]

        if new_point == 'end':
            possible_links = None
        else:
            possible_links = remaining_links.copy()

            if start_point == 'start':
                possible_links = [link for link in possible_links if 'start' not in link]

        new_path = path.copy()
        new_path.append(new_point)
        options.append((new_path, possible_links))
    
    return options


def find_paths(initial_options, previous_options=None):

    new_options = []
    for n, option in enumerate(initial_options.copy()):
        # print('n =',n)

        if option[1] is None:   # if the path has not ended
            continue

        new_option = find_ways(option[0], option[1])
        
        new_option = filter_options(new_option)
        
        new_options.extend(new_option)
        initial_options.remove(option)
    
    initial_options.extend(new_options)

    paths, possibilities = zip(*initial_options)
    # print('paths:\n', list(paths))
    # print('# of paths:', len(paths))
    print('New:\n', new_options)
    print('Pre:\n', previous_options)

    if all(possibilities) == None or len(new_options) == 0 or new_options == previous_options:
        for path in paths:
            print(path)
        print(len(paths))
        return initial_options
    else:
        find_paths(initial_options, new_options.copy())


def filter_options(options):

    for path, pos in options.copy():
        single_twice = None
        for cave in path:
            if cave.islower() and path.count(cave) == 2:
                if single_twice in (None, cave):
                    single_twice = cave
                    continue
                else:
                    # print('removing: ', '\n', (path))
                    options.remove((path, pos))
                    break
    
    print(len(options))
    return options


# paths = [item[0] for item in find_paths(options)]
find_paths(options)

In [45]:
class Path():

    def __init__(self, parent, cave_map, path=['start']):
        self.parent = parent
        self.cave_map = cave_map
        self.path = path
        self.parent.append(self)
    

    def find_paths(self):
        
        start_point = self.path[-1]

        self.check_visits()

        split_links = [link.split('-') for link in self.cave_map.copy()]
        relevant_links = [link for link in split_links if start_point in link]
    
        for link in relevant_links:
            print('link:', link)

            new_path = self.extend_path(link, start_point)

            if new_path[-1] == 'end':
                cave_map = None
            else:
                cave_map = self.cave_map

            if start_point == 'start':
                cave_map = [link for link in cave_map()
                            if 'start' not in link]

            if self in self.parent:
                self.parent.remove(self)
            # print(new_path)
            Path(self.parent, cave_map, new_path)


    def extend_path(self, link, start_point):

        # print('link:', link)
        # print('start_point:', start_point)
        link.remove(start_point)
        # print('new_point:', new_point)
        new_point = link[0]

        new_path = self.path.copy()
        new_path.append(new_point)

        return new_path


    def check_visits(self):

        lower_points = [point for point in self.path if point.islower()]
        lower_points = Counter(lower_points)

        double_visit = None
        for point, count in lower_points.items():
            if count == 2:
                double_visit = point
        
        if double_visit:
            for link in self.cave_map.copy():
                p1, p2 = link.split('-')

                if lower_points[p1] >=1 or lower_points[p2] >= 1:
                    self.cave_map.remove(link)


paths = []

a = Path(paths, test_data1)
paths

In [46]:

while True:
    ended = True
    for path in paths.copy():
        if 'end' not in path.path:
            path.find_paths()
            print(path.path)
            print(path.cave_map)
            ended = False

    if ended:
        break


# for path in paths:
#     print(path.path)

['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']
['start']
['A-end']


KeyboardInterrupt: 

In [33]:
paths

In [13]:
paths, _ = zip(*options)
for path in paths:
    print(path)

NameError: name 'options' is not defined

In [ ]:
# Part 2 answer